In [2]:
import sqlite3 as sql
conn = sql.connect("./travel.sqlite")

In [3]:
import pandas as pd

airports_df = pd.read_sql_query("select * from airports_data", conn)
airports_df.to_csv("./airports.csv")
airports_df.head()

,airport_code,airport_name,city,coordinates,timezone
0,YKS,"{""en"": ""Yakutsk Airport"", ""ru"": ""Якутск""}","{""en"": ""Yakutsk"", ""ru"": ""Якутск""}","(129.77099609375,62.0932998657226562)",Asia/Yakutsk
1,MJZ,"{""en"": ""Mirny Airport"", ""ru"": ""Мирный""}","{""en"": ""Mirnyj"", ""ru"": ""Мирный""}","(114.03900146484375,62.534698486328125)",Asia/Yakutsk
2,KHV,"{""en"": ""Khabarovsk-Novy Airport"", ""ru"": ""Хабар...","{""en"": ""Khabarovsk"", ""ru"": ""Хабаровск""}","(135.18800354004,48.5279998779300001)",Asia/Vladivostok
3,PKC,"{""en"": ""Yelizovo Airport"", ""ru"": ""Елизово""}","{""en"": ""Petropavlovsk"", ""ru"": ""Петропавловск-К...","(158.453994750976562,53.1679000854492188)",Asia/Kamchatka
4,UUS,"{""en"": ""Yuzhno-Sakhalinsk Airport"", ""ru"": ""Хом...","{""en"": ""Yuzhno-Sakhalinsk"", ""ru"": ""Южно-Сахали...","(142.718002319335938,46.8886985778808594)",Asia/Sakhalin


In [4]:
#Finish gathering all the individual tables I need
bookings_df = pd.read_sql_query("select * from bookings", conn)
bookings_df.to_csv("./bookings.csv")

flights_df = pd.read_sql_query("select * from flights", conn)
flights_df.to_csv("./flights.csv")

tckflights_df = pd.read_sql_query("select * from ticket_flights", conn)
tckflights_df.to_csv("./tckflights.csv")

tickets_df = pd.read_sql_query("select * from tickets", conn)
tickets_df.to_csv("./tickets.csv")

In [5]:
#Step 2 is to merge the tables that can easily be merged
#Start with bookings and tickets
booktickdf = pd.merge(bookings_df, tickets_df, on='book_ref') 
booktickdf.to_csv('./booktick.csv', index=False)

In [6]:
tkflightdf = pd.merge(booktickdf, tckflights_df, on='ticket_no') 
tkflightdf.to_csv('./tkflight.csv', index=False)

In [7]:
final_df = pd.merge(tkflightdf, flights_df, on='flight_id') 
final_df.to_csv('./flightprices.csv', index=False)
final_df.head()

,book_ref,book_date,total_amount,ticket_no,passenger_id,flight_id,fare_conditions,amount,flight_no,scheduled_departure,scheduled_arrival,departure_airport,arrival_airport,status,aircraft_code,actual_departure,actual_arrival
0,00000F,2017-07-05 03:12:00+03,265700,0005435838975,1708 262537,5995,Economy,66400,PG0703,2017-07-16 17:15:00+03,2017-07-17 02:00:00+03,SVO,UUS,Arrived,319,2017-07-16 17:19:00+03,2017-07-17 01:57:00+03
1,00000F,2017-07-05 03:12:00+03,265700,0005435838975,1708 262537,18058,Business,199300,PG0704,2017-07-24 10:45:00+03,2017-07-24 19:30:00+03,UUS,SVO,Arrived,319,2017-07-24 10:47:00+03,2017-07-24 19:26:00+03
2,000012,2017-07-14 09:02:00+03,37900,0005432527326,9091 269355,7737,Economy,14000,PG0224,2017-07-28 17:05:00+03,2017-07-28 18:50:00+03,SVO,AER,Arrived,773,2017-07-28 17:08:00+03,2017-07-28 18:54:00+03
3,000012,2017-07-14 09:02:00+03,37900,0005432527326,9091 269355,30620,Comfort,23900,PG0013,2017-08-09 18:15:00+03,2017-08-09 20:00:00+03,AER,SVO,Arrived,773,2017-08-09 18:17:00+03,2017-08-09 20:03:00+03
4,000068,2017-08-15 14:27:00+03,18100,0005432293273,5895 674437,1039,Economy,10200,PG0517,2017-08-28 10:30:00+03,2017-08-28 11:50:00+03,DME,SCW,Scheduled,763,\N,\N


In [8]:
conn.close() #don't need this anymore so just to be safe

#Drop columns I don't need
#Some were used for merging and no longer needed- ticket_no is unique key
#Others I'm simply not interested in for my analysis
clean_df = final_df.drop(['book_ref', 'total_amount', 'passenger_id', 'flight_id', 'flight_no', 'status', 'aircraft_code', 'actual_departure', 'actual_arrival'], axis=1)
clean_df.head()

,book_date,ticket_no,fare_conditions,amount,scheduled_departure,scheduled_arrival,departure_airport,arrival_airport
0,2017-07-05 03:12:00+03,0005435838975,Economy,66400,2017-07-16 17:15:00+03,2017-07-17 02:00:00+03,SVO,UUS
1,2017-07-05 03:12:00+03,0005435838975,Business,199300,2017-07-24 10:45:00+03,2017-07-24 19:30:00+03,UUS,SVO
2,2017-07-14 09:02:00+03,0005432527326,Economy,14000,2017-07-28 17:05:00+03,2017-07-28 18:50:00+03,SVO,AER
3,2017-07-14 09:02:00+03,0005432527326,Comfort,23900,2017-08-09 18:15:00+03,2017-08-09 20:00:00+03,AER,SVO
4,2017-08-15 14:27:00+03,0005432293273,Economy,10200,2017-08-28 10:30:00+03,2017-08-28 11:50:00+03,DME,SCW


In [9]:
#Rename columns for readability and to grab data using airports table as a reference
#(First I will prepare to get the data from departure_airport)
clean_df.rename(columns={"book_date": "Booking Date", "fare_conditions": "Booking Class", "amount": "Price", "scheduled_departure": "Departure", "scheduled_arrival": "Arrival", "departure_airport": "airport_code"}, inplace=True)
clean_df.head()

,Booking Date,ticket_no,Booking Class,Price,Departure,Arrival,airport_code,arrival_airport
0,2017-07-05 03:12:00+03,0005435838975,Economy,66400,2017-07-16 17:15:00+03,2017-07-17 02:00:00+03,SVO,UUS
1,2017-07-05 03:12:00+03,0005435838975,Business,199300,2017-07-24 10:45:00+03,2017-07-24 19:30:00+03,UUS,SVO
2,2017-07-14 09:02:00+03,0005432527326,Economy,14000,2017-07-28 17:05:00+03,2017-07-28 18:50:00+03,SVO,AER
3,2017-07-14 09:02:00+03,0005432527326,Comfort,23900,2017-08-09 18:15:00+03,2017-08-09 20:00:00+03,AER,SVO
4,2017-08-15 14:27:00+03,0005432293273,Economy,10200,2017-08-28 10:30:00+03,2017-08-28 11:50:00+03,DME,SCW


In [10]:
clean_df = pd.merge(clean_df, airports_df, on='airport_code') 
clean_df.to_csv('./flightprices.csv', index=False)
clean_df.head()

,Booking Date,ticket_no,Booking Class,Price,Departure,Arrival,airport_code,arrival_airport,airport_name,city,coordinates,timezone
0,2017-07-05 03:12:00+03,0005435838975,Economy,66400,2017-07-16 17:15:00+03,2017-07-17 02:00:00+03,SVO,UUS,"{""en"": ""Sheremetyevo International Airport"", ""...","{""en"": ""Moscow"", ""ru"": ""Москва""}","(37.4146000000000001,55.9725990000000024)",Europe/Moscow
1,2017-07-05 03:12:00+03,0005435838975,Business,199300,2017-07-24 10:45:00+03,2017-07-24 19:30:00+03,UUS,SVO,"{""en"": ""Yuzhno-Sakhalinsk Airport"", ""ru"": ""Хом...","{""en"": ""Yuzhno-Sakhalinsk"", ""ru"": ""Южно-Сахали...","(142.718002319335938,46.8886985778808594)",Asia/Sakhalin
2,2017-07-14 09:02:00+03,0005432527326,Economy,14000,2017-07-28 17:05:00+03,2017-07-28 18:50:00+03,SVO,AER,"{""en"": ""Sheremetyevo International Airport"", ""...","{""en"": ""Moscow"", ""ru"": ""Москва""}","(37.4146000000000001,55.9725990000000024)",Europe/Moscow
3,2017-07-14 09:02:00+03,0005432527326,Comfort,23900,2017-08-09 18:15:00+03,2017-08-09 20:00:00+03,AER,SVO,"{""en"": ""Sochi International Airport"", ""ru"": ""С...","{""en"": ""Sochi"", ""ru"": ""Сочи""}","(39.9566001892089986,43.4499015808110016)",Europe/Moscow
4,2017-08-15 14:27:00+03,0005432293273,Economy,10200,2017-08-28 10:30:00+03,2017-08-28 11:50:00+03,DME,SCW,"{""en"": ""Domodedovo International Airport"", ""ru...","{""en"": ""Moscow"", ""ru"": ""Москва""}","(37.9062995910644531,55.4087982177734375)",Europe/Moscow


In [11]:
#Remove unneeded columns and prepare to get arrival_airport references
clean_df.drop(['airport_name', 'city', 'timezone'], axis=1, inplace=True)
clean_df.rename(columns={"airport_code": "Departure Airport", "arrival_airport": "airport_code"}, inplace=True)
clean_df.head()

,Booking Date,ticket_no,Booking Class,Price,Departure,Arrival,Departure Airport,airport_code,coordinates
0,2017-07-05 03:12:00+03,0005435838975,Economy,66400,2017-07-16 17:15:00+03,2017-07-17 02:00:00+03,SVO,UUS,"(37.4146000000000001,55.9725990000000024)"
1,2017-07-05 03:12:00+03,0005435838975,Business,199300,2017-07-24 10:45:00+03,2017-07-24 19:30:00+03,UUS,SVO,"(142.718002319335938,46.8886985778808594)"
2,2017-07-14 09:02:00+03,0005432527326,Economy,14000,2017-07-28 17:05:00+03,2017-07-28 18:50:00+03,SVO,AER,"(37.4146000000000001,55.9725990000000024)"
3,2017-07-14 09:02:00+03,0005432527326,Comfort,23900,2017-08-09 18:15:00+03,2017-08-09 20:00:00+03,AER,SVO,"(39.9566001892089986,43.4499015808110016)"
4,2017-08-15 14:27:00+03,0005432293273,Economy,10200,2017-08-28 10:30:00+03,2017-08-28 11:50:00+03,DME,SCW,"(37.9062995910644531,55.4087982177734375)"


In [12]:
#forgot to rename coordinates
clean_df.rename(columns={"coordinates": "Departure Coordinates"}, inplace=True)
clean_df.head()

,Booking Date,ticket_no,Booking Class,Price,Departure,Arrival,Departure Airport,airport_code,Departure Coordinates
0,2017-07-05 03:12:00+03,0005435838975,Economy,66400,2017-07-16 17:15:00+03,2017-07-17 02:00:00+03,SVO,UUS,"(37.4146000000000001,55.9725990000000024)"
1,2017-07-05 03:12:00+03,0005435838975,Business,199300,2017-07-24 10:45:00+03,2017-07-24 19:30:00+03,UUS,SVO,"(142.718002319335938,46.8886985778808594)"
2,2017-07-14 09:02:00+03,0005432527326,Economy,14000,2017-07-28 17:05:00+03,2017-07-28 18:50:00+03,SVO,AER,"(37.4146000000000001,55.9725990000000024)"
3,2017-07-14 09:02:00+03,0005432527326,Comfort,23900,2017-08-09 18:15:00+03,2017-08-09 20:00:00+03,AER,SVO,"(39.9566001892089986,43.4499015808110016)"
4,2017-08-15 14:27:00+03,0005432293273,Economy,10200,2017-08-28 10:30:00+03,2017-08-28 11:50:00+03,DME,SCW,"(37.9062995910644531,55.4087982177734375)"


In [13]:
#Now get the arrival airport coordinates with the same process
clean_df = pd.merge(clean_df, airports_df, on='airport_code') 
clean_df.to_csv('./flightprices.csv', index=False)
clean_df.head()

,Booking Date,ticket_no,Booking Class,Price,Departure,Arrival,Departure Airport,airport_code,Departure Coordinates,airport_name,city,coordinates,timezone
0,2017-07-05 03:12:00+03,0005435838975,Economy,66400,2017-07-16 17:15:00+03,2017-07-17 02:00:00+03,SVO,UUS,"(37.4146000000000001,55.9725990000000024)","{""en"": ""Yuzhno-Sakhalinsk Airport"", ""ru"": ""Хом...","{""en"": ""Yuzhno-Sakhalinsk"", ""ru"": ""Южно-Сахали...","(142.718002319335938,46.8886985778808594)",Asia/Sakhalin
1,2017-07-05 03:12:00+03,0005435838975,Business,199300,2017-07-24 10:45:00+03,2017-07-24 19:30:00+03,UUS,SVO,"(142.718002319335938,46.8886985778808594)","{""en"": ""Sheremetyevo International Airport"", ""...","{""en"": ""Moscow"", ""ru"": ""Москва""}","(37.4146000000000001,55.9725990000000024)",Europe/Moscow
2,2017-07-14 09:02:00+03,0005432527326,Economy,14000,2017-07-28 17:05:00+03,2017-07-28 18:50:00+03,SVO,AER,"(37.4146000000000001,55.9725990000000024)","{""en"": ""Sochi International Airport"", ""ru"": ""С...","{""en"": ""Sochi"", ""ru"": ""Сочи""}","(39.9566001892089986,43.4499015808110016)",Europe/Moscow
3,2017-07-14 09:02:00+03,0005432527326,Comfort,23900,2017-08-09 18:15:00+03,2017-08-09 20:00:00+03,AER,SVO,"(39.9566001892089986,43.4499015808110016)","{""en"": ""Sheremetyevo International Airport"", ""...","{""en"": ""Moscow"", ""ru"": ""Москва""}","(37.4146000000000001,55.9725990000000024)",Europe/Moscow
4,2017-08-15 14:27:00+03,0005432293273,Economy,10200,2017-08-28 10:30:00+03,2017-08-28 11:50:00+03,DME,SCW,"(37.9062995910644531,55.4087982177734375)","{""en"": ""Syktyvkar Airport"", ""ru"": ""Сыктывкар""}","{""en"": ""Syktyvkar"", ""ru"": ""Сыктывкар""}","(50.8451004028320312,61.6469993591308594)",Europe/Moscow


In [14]:
#Clean the arrival airport data
clean_df.drop(['airport_name', 'city', 'timezone'], axis=1, inplace=True)
clean_df.rename(columns={"airport_code": "Arrival Airport", "coordinates": "Arrival Coordinates"}, inplace=True)
clean_df.head()

,Booking Date,ticket_no,Booking Class,Price,Departure,Arrival,Departure Airport,Arrival Airport,Departure Coordinates,Arrival Coordinates
0,2017-07-05 03:12:00+03,0005435838975,Economy,66400,2017-07-16 17:15:00+03,2017-07-17 02:00:00+03,SVO,UUS,"(37.4146000000000001,55.9725990000000024)","(142.718002319335938,46.8886985778808594)"
1,2017-07-05 03:12:00+03,0005435838975,Business,199300,2017-07-24 10:45:00+03,2017-07-24 19:30:00+03,UUS,SVO,"(142.718002319335938,46.8886985778808594)","(37.4146000000000001,55.9725990000000024)"
2,2017-07-14 09:02:00+03,0005432527326,Economy,14000,2017-07-28 17:05:00+03,2017-07-28 18:50:00+03,SVO,AER,"(37.4146000000000001,55.9725990000000024)","(39.9566001892089986,43.4499015808110016)"
3,2017-07-14 09:02:00+03,0005432527326,Comfort,23900,2017-08-09 18:15:00+03,2017-08-09 20:00:00+03,AER,SVO,"(39.9566001892089986,43.4499015808110016)","(37.4146000000000001,55.9725990000000024)"
4,2017-08-15 14:27:00+03,0005432293273,Economy,10200,2017-08-28 10:30:00+03,2017-08-28 11:50:00+03,DME,SCW,"(37.9062995910644531,55.4087982177734375)","(50.8451004028320312,61.6469993591308594)"


In [15]:
#Actually I only need the coordinates so drop the airport codes as well
clean_df.drop(['Departure Airport', 'Arrival Airport'], axis=1, inplace=True)

In [18]:
#Calculating a flight distance column
from haversine import haversine as hs

#First off coordinates should not have parentheses
#Also there are out of range values which seem to be swapped
def clean_coordinates(coord_str):
    # Remove parentheses and convert to floats
    coord = eval(coord_str.strip("()"))
    lat, lon = coord
    
    # Check and swap if latitude is out of bounds
    if abs(lat) > 90:
        lat, lon = lon, lat  # Swap latitude and longitude if lat is out of range
    return (lat, lon)

# Apply the cleaning function to both columns
clean_df["Departure Coordinates"] = clean_df["Departure Coordinates"].apply(clean_coordinates)
clean_df["Arrival Coordinates"] = clean_df["Arrival Coordinates"].apply(clean_coordinates)

# Calculate distances
clean_df["Distance"] = clean_df.apply(lambda row: hs(row["Departure Coordinates"], row["Arrival Coordinates"]), axis=1)

clean_df.head()

,Booking Date,ticket_no,Booking Class,Price,Departure,Arrival,Departure Coordinates,Arrival Coordinates,Distance
0,2017-07-05 03:12:00+03,0005435838975,Economy,66400,2017-07-16 17:15:00+03,2017-07-17 02:00:00+03,"(37.4146, 55.972599)","(46.88869857788086, 142.71800231933594)",6858.727758
1,2017-07-05 03:12:00+03,0005435838975,Business,199300,2017-07-24 10:45:00+03,2017-07-24 19:30:00+03,"(46.88869857788086, 142.71800231933594)","(37.4146, 55.972599)",6858.727758
2,2017-07-14 09:02:00+03,0005432527326,Economy,14000,2017-07-28 17:05:00+03,2017-07-28 18:50:00+03,"(37.4146, 55.972599)","(39.956600189209, 43.449901580811)",1122.017357
3,2017-07-14 09:02:00+03,0005432527326,Comfort,23900,2017-08-09 18:15:00+03,2017-08-09 20:00:00+03,"(39.956600189209, 43.449901580811)","(37.4146, 55.972599)",1122.017357
4,2017-08-15 14:27:00+03,0005432293273,Economy,10200,2017-08-28 10:30:00+03,2017-08-28 11:50:00+03,"(37.90629959106445, 55.40879821777344)","(50.84510040283203, 61.64699935913086)",1520.392126


In [20]:
#Coordinates no longer needed
clean_df.drop(['Departure Coordinates', 'Arrival Coordinates'], axis=1, inplace=True)
clean_df.head()

,Booking Date,ticket_no,Booking Class,Price,Departure,Arrival,Distance
0,2017-07-05 03:12:00+03,0005435838975,Economy,66400,2017-07-16 17:15:00+03,2017-07-17 02:00:00+03,6858.727758
1,2017-07-05 03:12:00+03,0005435838975,Business,199300,2017-07-24 10:45:00+03,2017-07-24 19:30:00+03,6858.727758
2,2017-07-14 09:02:00+03,0005432527326,Economy,14000,2017-07-28 17:05:00+03,2017-07-28 18:50:00+03,1122.017357
3,2017-07-14 09:02:00+03,0005432527326,Comfort,23900,2017-08-09 18:15:00+03,2017-08-09 20:00:00+03,1122.017357
4,2017-08-15 14:27:00+03,0005432293273,Economy,10200,2017-08-28 10:30:00+03,2017-08-28 11:50:00+03,1520.392126


In [26]:
#Calculate lead time between booking and departure as # of days
from datetime import datetime as dt

# Remove the timezone part and convert to datetime
clean_df['Booking Date'] = pd.to_datetime(clean_df['Booking Date'].str[:-3], format="%Y-%m-%d %H:%M:%S")
clean_df['Departure'] = pd.to_datetime(clean_df['Departure'].str[:-3], format="%Y-%m-%d %H:%M:%S")

# Calculate the difference in days
clean_df['Lead Time'] = (clean_df['Departure'] - clean_df['Booking Date']).dt.days

# Display the result
print(clean_df.head())

         Booking Date      ticket_no Booking Class   Price  \
0 2017-07-05 03:12:00  0005435838975       Economy   66400   
1 2017-07-05 03:12:00  0005435838975      Business  199300   
2 2017-07-14 09:02:00  0005432527326       Economy   14000   
3 2017-07-14 09:02:00  0005432527326       Comfort   23900   
4 2017-08-15 14:27:00  0005432293273       Economy   10200   

            Departure                 Arrival     Distance  Lead Time  
0 2017-07-16 17:15:00  2017-07-17 02:00:00+03  6858.727758         11  
1 2017-07-24 10:45:00  2017-07-24 19:30:00+03  6858.727758         19  
2 2017-07-28 17:05:00  2017-07-28 18:50:00+03  1122.017357         14  
3 2017-08-09 18:15:00  2017-08-09 20:00:00+03  1122.017357         26  
4 2017-08-28 10:30:00  2017-08-28 11:50:00+03  1520.392126         12  


In [28]:
clean_df.rename(columns={"Lead Time": "Lead Time (Days)"}, inplace=True)
clean_df.head()

,Booking Date,ticket_no,Booking Class,Price,Departure,Arrival,Distance,Lead Time (Days)
0,2017-07-05 03:12:00,0005435838975,Economy,66400,2017-07-16 17:15:00,2017-07-17 02:00:00+03,6858.727758,11
1,2017-07-05 03:12:00,0005435838975,Business,199300,2017-07-24 10:45:00,2017-07-24 19:30:00+03,6858.727758,19
2,2017-07-14 09:02:00,0005432527326,Economy,14000,2017-07-28 17:05:00,2017-07-28 18:50:00+03,1122.017357,14
3,2017-07-14 09:02:00,0005432527326,Comfort,23900,2017-08-09 18:15:00,2017-08-09 20:00:00+03,1122.017357,26
4,2017-08-15 14:27:00,0005432293273,Economy,10200,2017-08-28 10:30:00,2017-08-28 11:50:00+03,1520.392126,12


In [32]:
# Extract weekday, month name, and hour of day from booking and departure dates
clean_df['Booking Weekday'] = clean_df['Booking Date'].dt.day_name()
clean_df['Booking Month'] = clean_df['Booking Date'].dt.month_name()
clean_df['Booking Hour'] = clean_df['Booking Date'].dt.hour

clean_df['Departure Weekday'] = clean_df['Departure'].dt.day_name()
clean_df['Departure Month'] = clean_df['Departure'].dt.month_name()
clean_df['Departure Hour'] = clean_df['Departure'].dt.hour

clean_df.head()

,Booking Date,ticket_no,Booking Class,Price,Departure,Arrival,Distance,Lead Time (Days),Booking Weekday,Booking Month,Booking Hour,Departure Weekday,Departure Month,Departure Hour
0,2017-07-05 03:12:00,0005435838975,Economy,66400,2017-07-16 17:15:00,2017-07-17 02:00:00+03,6858.727758,11,Wednesday,July,3,Sunday,July,17
1,2017-07-05 03:12:00,0005435838975,Business,199300,2017-07-24 10:45:00,2017-07-24 19:30:00+03,6858.727758,19,Wednesday,July,3,Monday,July,10
2,2017-07-14 09:02:00,0005432527326,Economy,14000,2017-07-28 17:05:00,2017-07-28 18:50:00+03,1122.017357,14,Friday,July,9,Friday,July,17
3,2017-07-14 09:02:00,0005432527326,Comfort,23900,2017-08-09 18:15:00,2017-08-09 20:00:00+03,1122.017357,26,Friday,July,9,Wednesday,August,18
4,2017-08-15 14:27:00,0005432293273,Economy,10200,2017-08-28 10:30:00,2017-08-28 11:50:00+03,1520.392126,12,Tuesday,August,14,Monday,August,10


In [34]:
#Now that I have all the columns I want for my research questions
#Drop the columns that I won't be using

clean_df.drop(['Booking Date', 'Departure', 'Arrival'], axis=1, inplace=True)
clean_df.head()

,ticket_no,Booking Class,Price,Distance,Lead Time (Days),Booking Weekday,Booking Month,Booking Hour,Departure Weekday,Departure Month,Departure Hour
0,0005435838975,Economy,66400,6858.727758,11,Wednesday,July,3,Sunday,July,17
1,0005435838975,Business,199300,6858.727758,19,Wednesday,July,3,Monday,July,10
2,0005432527326,Economy,14000,1122.017357,14,Friday,July,9,Friday,July,17
3,0005432527326,Comfort,23900,1122.017357,26,Friday,July,9,Wednesday,August,18
4,0005432293273,Economy,10200,1520.392126,12,Tuesday,August,14,Monday,August,10


In [36]:
#One thing I wanted to do for clarity for myself and my readers
#Convert km to miles and rubles to USD
#I will not save the old columns because I won't use them

conversion_rate = 0.010 #today's RUB -> USD conversion rate from Google
km_to_miles = 0.621371

clean_df['Price'] = clean_df['Price'] * conversion_rate
clean_df['Distance'] = clean_df['Distance'] * km_to_miles

clean_df.head()

,ticket_no,Booking Class,Price,Distance,Lead Time (Days),Booking Weekday,Booking Month,Booking Hour,Departure Weekday,Departure Month,Departure Hour
0,0005435838975,Economy,664.0,4261.814526,11,Wednesday,July,3,Sunday,July,17
1,0005435838975,Business,1993.0,4261.814526,19,Wednesday,July,3,Monday,July,10
2,0005432527326,Economy,140.0,697.189047,14,Friday,July,9,Friday,July,17
3,0005432527326,Comfort,239.0,697.189047,26,Friday,July,9,Wednesday,August,18
4,0005432293273,Economy,102.0,944.727576,12,Tuesday,August,14,Monday,August,10


In [38]:
#Now that I'm happy with my preprocessing, save to final CSV
clean_df.to_csv("./flightpricesFINAL.csv")